In [17]:
!pip install facenet-pytorch




     ---------------------------------------- 1.9/1.9 MB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 10.4 MB/s eta 0:00:00
     -------------------------------------- 198.6/198.6 MB 1.3 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 7.3 MB/s eta 0:00:00
     -------------------------------------- 179.6/179.6 kB 5.5 MB/s eta 0:00:00
     ------------------------------------- 536.2/536.2 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.0.0
    Uninstalling pillow-11.0.0:
      Successfully uninstalled pillow-11.0.0



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import os
import glob
import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
import cv2
from tqdm import tqdm

# Set paths to the dataset folders
real_videos_path = 'D:/test_dataset/real'
fake_videos_path = 'D:/test_dataset/fake'

# Output directories for processed videos
output_dir_real = 'D:/test_dataset/processed/real'
output_dir_fake = 'D:/test_dataset/processed/fake'

os.makedirs(output_dir_real, exist_ok=True)
os.makedirs(output_dir_fake, exist_ok=True)

# Initialize MTCNN (face detection) and InceptionResnetV1 (embeddings)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Function to extract frames from a video
def frame_extract(path):
    vid_obj = cv2.VideoCapture(path)
    success, image = True, None
    while success:
        success, image = vid_obj.read()
        if success:
            yield cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process frames and save face-only videos
def process_videos(video_paths, output_dir):
    for path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(path))
        if os.path.exists(out_path):
            print(f"File already exists: {out_path}")
            continue

        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MJPG'), 30, (112, 112))
        for idx, frame in enumerate(frame_extract(path)):
            if idx > 150:  # Limit to the first 150 frames
                break
            frame_pil = Image.fromarray(frame)
            boxes, _ = mtcnn.detect(frame_pil)
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box)
                    try:
                        cropped_face = frame[y1:y2, x1:x2]
                        resized_face = cv2.resize(cropped_face, (112, 112))
                        out.write(cv2.cvtColor(resized_face, cv2.COLOR_RGB2BGR))
                    except Exception as e:
                        print(f"Error processing face in {path}: {e}")
        out.release()

# Collect video files
video_files_real = glob.glob(os.path.join(real_videos_path, '*.mp4'))
video_files_fake = glob.glob(os.path.join(fake_videos_path, '*.mp4'))

# Process real and fake videos
print("Processing real videos...")
process_videos(video_files_real, output_dir_real)

print("Processing fake videos...")
process_videos(video_files_fake, output_dir_fake)


  0%|          | 0.00/107M [00:00<?, ?B/s]

Processing real videos...


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Error processing face in D:/test_dataset/real\0001e96803--623955e76e1ee5175569a12b__HONOR 10I .mp4: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error processing face in D:/test_dataset/real\0001e96803--623955e76e1ee5175569a12b__HONOR 10I .mp4: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error processing face in D:/test_dataset/real\0001e96803--623955e76e1ee5175569a12b__HONOR 10I .mp4: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'

Error processing face in D:/test_dataset/real\0001e96803--623955e76e1ee5175569a12b__HONOR 10I .mp4: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Ass

 50%|██████████████████████████████████████████                                          | 1/2 [00:34<00:34, 34.32s/it]

Error processing face in D:/test_dataset/real\0001e96803--623955e76e1ee5175569a12b__HONOR 10I .mp4: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:19<00:00, 39.85s/it]


Processing fake videos...


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:24<00:00, 42.13s/it]
